In [2]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import StratifiedKFold

In [3]:
train=pd.read_csv('../balanced_cleaned_trian_dataset.csv',index_col=0)
test_set=pd.read_csv('../testset.csv',index_col=0)

grouped = train.groupby("smoking")
df_0 = grouped.get_group(0)
df_1 = grouped.get_group(1)
df_0_half = df_0.sample(n=150, random_state=42)
df_1_half = df_1.sample(n=150, random_state=42)
df_half = pd.concat([df_0_half, df_1_half])

df_half = df_half.sample(frac=1, random_state=42).reset_index(drop=True)


In [4]:
train_copy=df_half.drop('id',axis=1)
X = train_copy.drop('smoking', axis=1)
y = train_copy['smoking']

test_set=test_set.drop('id',axis=1)
X_test_new= test_set.drop('smoking', axis=1)
y_test_new = test_set['smoking']

In [ ]:
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
import numpy as np

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_d = trial.suggest_int("n_d", 8, 32, step=4)       
    n_a = trial.suggest_int("n_a", 8, 32, step=4)       
    n_steps = trial.suggest_int("n_steps", 3, 6)        
    gamma = trial.suggest_float("gamma", 1.0, 1.5)      
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-5, 1e-3, log=True)
    lr = trial.suggest_float("lr", 1e-3, 5e-2, log=True)  
    max_epochs = trial.suggest_int("max_epochs", 20, 50)  

    scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled   = scaler.transform(X_val_fold)

        #  numpy
        X_train_fold = X_train_scaled.astype(np.float32)
        X_val_fold   = X_val_scaled.astype(np.float32)
        y_train_fold = y_train_fold.values.astype(np.int64)
        y_val_fold   = y_val_fold.values.astype(np.int64)

        model = TabNetClassifier(
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            lambda_sparse=lambda_sparse,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=lr),
            verbose=50,
            seed=42
        )

        model.fit(
            X_train_fold, y_train_fold,
            # eval_set=[(X_val_fold, y_val_fold)],
            # eval_metric=['accuracy'],
            max_epochs=max_epochs,
            # patience=5,
            batch_size=32,
            virtual_batch_size=32
        )

        y_pred_val = model.predict(X_val_fold)
        acc = accuracy_score(y_val_fold, y_pred_val)
        
        scores.append(acc)


    return np.mean(scores)


sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)

print("best_params:", study.best_params)
print("best_value:", study.best_value)

[I 2025-08-15 15:22:25,603] A new study created in memory with name: no-name-4fd042c0-7649-4761-9308-cda76c5d44a0
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.04213 | val_0_accuracy: 0.6     |  0:00:00s

Early stopping occurred at epoch 5 with best_epoch = 0 and best_val_0_accuracy = 0.6


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.96809 | val_0_accuracy: 0.6     |  0:00:00s

Early stopping occurred at epoch 6 with best_epoch = 1 and best_val_0_accuracy = 0.61667
epoch 0  | loss: 1.03628 | val_0_accuracy: 0.58333 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...


Early stopping occurred at epoch 12 with best_epoch = 7 and best_val_0_accuracy = 0.68333
epoch 0  | loss: 0.94933 | val_0_accuracy: 0.53333 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")



Early stopping occurred at epoch 18 with best_epoch = 13 and best_val_0_accuracy = 0.78333
epoch 0  | loss: 0.99966 | val_0_accuracy: 0.53333 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-08-15 15:23:11,396] Trial 19 finished with value: 0.7133333333333333 and parameters: {'n_d': 24, 'n_a': 28, 'n_steps': 4, 'gamma': 1.0853347197353578, 'lambda_sparse': 0.00011982448134112166, 'lr': 0.005266899306365387, 'max_epochs': 46}. Best is trial 11 with value: 0.7466666666666667.



Early stopping occurred at epoch 16 with best_epoch = 11 and best_val_0_accuracy = 0.71667
best_params: {'n_d': 24, 'n_a': 8, 'n_steps': 4, 'gamma': 1.140380477119852, 'lambda_sparse': 1.0264309282867028e-05, 'lr': 0.037624249790989446, 'max_epochs': 43}
best_value: 0.7466666666666667


In [5]:

scaler = StandardScaler()
X_full_scaled      = scaler.fit_transform(X)           
X_test_new_scaled  = scaler.transform(X_test_new)

X_full_scaled_np       = X_full_scaled.astype(np.float32)
y_full_np              = y.values.astype(np.int64)
X_test_new_scaled_np   = X_test_new_scaled.astype(np.float32)
y_test_new_np          = y_test_new.values.astype(np.int64)


#best_params: {'n_d': 24, 'n_a': 8, 'n_steps': 4, 'gamma': 1.140380477119852, 'lambda_sparse': 1.0264309282867028e-05, 'lr': 0.037624249790989446, 'max_epochs': 43}
#best_value: 0.7466666666666667

tabnet_final = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.037624249790989446),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=10,
    n_d=24, n_a=8, n_steps=4, gamma=1.140380477119852,
    lambda_sparse=1.0264309282867028e-05,
    seed=42
)

tabnet_final.fit(
    X_train=X_full_scaled_np, y_train=y_full_np,
    batch_size=16,
    virtual_batch_size=8,
    max_epochs=43,
    num_workers=0
)



y_train_pred = tabnet_final.predict(X_full_scaled_np)
train_acc = accuracy_score(y_full_np, y_train_pred)
print(f"TabNet Train Accuracy (full-train): {train_acc:.4f}")

y_test_pred = tabnet_final.predict(X_test_new_scaled_np)
test_acc = accuracy_score(y_test_new_np, y_test_pred)
print(f"\nTabNet Test Accuracy: {test_acc:.4f}")
print("Test Classification Report:\n", classification_report(y_test_new_np, y_test_pred, digits=4))


epoch 0  | loss: 1.19864 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 10 | loss: 0.58477 |  0:00:00s
epoch 20 | loss: 0.58591 |  0:00:01s
epoch 30 | loss: 0.59016 |  0:00:02s
epoch 40 | loss: 0.54368 |  0:00:03s
TabNet Train Accuracy (full-train): 0.7533

TabNet Test Accuracy: 0.7265
Test Classification Report:
               precision    recall  f1-score   support

           0     0.7314    0.7160    0.7236      1000
           1     0.7218    0.7370    0.7293      1000

    accuracy                         0.7265      2000
   macro avg     0.7266    0.7265    0.7265      2000
weighted avg     0.7266    0.7265    0.7265      2000



In [6]:

X_train_f, X_val_f, y_train_f, y_val_f = train_test_split(
    X, y, test_size=0.2, random_state=42
)


scaler = StandardScaler()
X_train_f_scaled = scaler.fit_transform(X_train_f)
X_val_f_scaled   = scaler.transform(X_val_f)
X_test_new_scaled  = scaler.transform(X_test_new)


X_train_scaled_np = X_train_f_scaled.astype(np.float32)
y_train_np = y_train_f.values.astype(np.int64)

X_val_scaled_np = X_val_f_scaled.astype(np.float32)
y_val_np = y_val_f.values.astype(np.int64)

X_test_new_scaled_np = X_test_new_scaled.astype(np.float32)
y_test_new_np = y_test_new.values.astype(np.int64)

#best_params: {'n_d': 24, 'n_a': 8, 'n_steps': 4, 'gamma': 1.140380477119852, 'lambda_sparse': 1.0264309282867028e-05, 'lr': 0.037624249790989446, 'max_epochs': 43}
#best_value: 0.7466666666666667

tabnet_model = TabNetClassifier(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=0.037624249790989446),
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=10,
    n_d=24, n_a=8, n_steps=4, gamma=1.140380477119852,
    lambda_sparse=1.0264309282867028e-05,
    seed=42
)


tabnet_model.fit(
    X_train=X_train_scaled_np, y_train=y_train_np,
    eval_set=[(X_val_scaled_np, y_val_np)],
    eval_metric=['accuracy'],
    patience=5,
    batch_size=16,
    virtual_batch_size=8,
    max_epochs=43,
    num_workers=0
)

y_train_pred = tabnet_model.predict(X_train_scaled_np)
train_acc = accuracy_score(y_train_np, y_train_pred)

print(f"TabNet Train Accuracy: {train_acc:.4f}")
print(classification_report(y_train_np, y_train_pred, digits=4))


y_pred = tabnet_model.predict(X_test_new_scaled_np)
acc = accuracy_score(y_test_new_np, y_pred)

print(f"\nTabNet Test Accuracy: {acc:.4f}")
print(classification_report(y_test_new_np, y_pred, digits=4))

/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.08139 | val_0_accuracy: 0.51667 |  0:00:00s

Early stopping occurred at epoch 9 with best_epoch = 4 and best_val_0_accuracy = 0.75
TabNet Train Accuracy: 0.6250
              precision    recall  f1-score   support

           0     0.6371    0.6371    0.6371       124
           1     0.6121    0.6121    0.6121       116

    accuracy                         0.6250       240
   macro avg     0.6246    0.6246    0.6246       240
weighted avg     0.6250    0.6250    0.6250       240



/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



TabNet Test Accuracy: 0.6095
              precision    recall  f1-score   support

           0     0.6048    0.6320    0.6181      1000
           1     0.6147    0.5870    0.6005      1000

    accuracy                         0.6095      2000
   macro avg     0.6097    0.6095    0.6093      2000
weighted avg     0.6097    0.6095    0.6093      2000



In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled   = scaler.transform(X_val_fold)

    
    X_train_fold = X_train_scaled.astype(np.float32)
    X_val_fold   = X_val_scaled.astype(np.float32)
    y_train_fold = y_train_fold.values.astype(np.int64)
    y_val_fold   = y_val_fold.values.astype(np.int64)

    model = TabNetClassifier(
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=0.037624249790989446),
        scheduler_params={"step_size":10, "gamma":0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        verbose=10,
        n_d=24, n_a=8, n_steps=4, gamma=1.140380477119852,
        lambda_sparse=1.0264309282867028e-05,
        seed=42
    )

    model.fit(
        X_train=X_train_fold, y_train=y_train_fold,
        batch_size=16,
        virtual_batch_size=8,
        max_epochs=43,
        num_workers=0
    )

    y_pred_val = model.predict(X_val_fold)
    acc = accuracy_score(y_val_fold, y_pred_val)

    y_train_pred = model.predict(X_train_fold)
    train_acc = accuracy_score(y_train_fold, y_train_pred)

    
    print(f"TabNet Train Accuracy: {train_acc:.4f}")


    print(f"\n TabNet Test Accuracy: {acc:.4f}")
    


   




/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.00681 |  0:00:00s
epoch 10 | loss: 0.62241 |  0:00:00s
epoch 20 | loss: 0.59472 |  0:00:01s
epoch 30 | loss: 0.56915 |  0:00:02s
epoch 40 | loss: 0.56613 |  0:00:02s
TabNet Train Accuracy: 0.7167

 TabNet Test Accuracy: 0.6333


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.29148 |  0:00:00s
epoch 10 | loss: 0.59539 |  0:00:00s
epoch 20 | loss: 0.59412 |  0:00:01s
epoch 30 | loss: 0.54214 |  0:00:01s
epoch 40 | loss: 0.55593 |  0:00:02s
TabNet Train Accuracy: 0.7583

 TabNet Test Accuracy: 0.6833


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.23012 |  0:00:00s
epoch 10 | loss: 0.59863 |  0:00:00s
epoch 20 | loss: 0.63106 |  0:00:01s
epoch 30 | loss: 0.5743  |  0:00:02s
epoch 40 | loss: 0.58065 |  0:00:02s
TabNet Train Accuracy: 0.7875

 TabNet Test Accuracy: 0.6500


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.24591 |  0:00:00s
epoch 10 | loss: 0.6332  |  0:00:00s
epoch 20 | loss: 0.64025 |  0:00:01s
epoch 30 | loss: 0.61913 |  0:00:02s
epoch 40 | loss: 0.57577 |  0:00:02s
TabNet Train Accuracy: 0.6958

 TabNet Test Accuracy: 0.7333


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.08722 |  0:00:00s
epoch 10 | loss: 0.65089 |  0:00:00s
epoch 20 | loss: 0.59953 |  0:00:01s
epoch 30 | loss: 0.56939 |  0:00:02s
epoch 40 | loss: 0.61032 |  0:00:02s
TabNet Train Accuracy: 0.7208

 TabNet Test Accuracy: 0.8000
